# Programming Exercise: Lorenz Curves and Gini Coefficients

-----

#### Chase Coleman and John Stachurski

#### Prepared for the QuantEcon ICD Computational Workshop (March 2024)

This notebook contains some programming exercises related to the Lorenz curve
and the Gini coefficient, which are often used to study inequality.

Your task will be to compute these curves and values, replicating functionality
that already exists in `quantecon`.

Uncomment the following if necessary

In [ ]:
#!pip install quantecon 

We use the following imports.

In [ ]:
import numba
import numpy as np
import matplotlib.pyplot as plt
import quantecon as qe

## Preamble: The Lorenz curve and Gini coefficient


### Definition

Let $w_1, \ldots, w_n$ be a sample of observations of wealth (or income, or consumption, or firm sizes, etc.) in a population.

Suppose the sample has been sorted from smallest to largest.

The Lorenz curve takes this sample and produces a curve $L$.

To create it we first generate data points $(x_i, y_i)_{i=0}^n$  according to

\begin{equation*}
    x_0 = y_0 = 0
    \qquad \text{and, for $i \geq 1$,} \quad
    x_i = \frac{i}{n},
    \qquad
    y_i =
       \frac{\sum_{j \leq i} w_j}{\sum_{j \leq n} w_j}  
\end{equation*}

Now the Lorenz curve $L$ is formed from these data points using interpolation.

The meaning of the statement $y = L(x)$ is that the lowest $(100 \times x)$\% of
people have $(100 \times y)$\% of all wealth.

* if $x=0.5$ and $y=0.1$, then the bottom 50% of the population
  owns 10% of the wealth.

### Using QuantEcon's routine

In the next figure, we generate $n=2000$ draws from a lognormal
distribution and treat these draws as our population.  

We then generate the Lorenz curve using a routine from `quantecon`.

The straight line ($x=L(x)$ for all $x$) corresponds to perfect equality.  

The lognormal draws produce a less equal distribution.

In [ ]:
n = 2000
sample = np.exp(np.random.randn(n))       # Lognormal sample
x, y = qe.lorenz_curve(sample)            # QuantEcon routine (no need to sort)

fig, ax = plt.subplots()
ax.plot(x, y, label=f'lognormal sample', lw=2)
ax.plot(x, x, label='equality', lw=2)
ax.legend(fontsize=12)
ax.set_ylim((0, 1))
ax.set_xlim((0, 1))
j = 1600  # dashed lines for j-th element
ax.vlines(x[j], [0.0], y[j], alpha=0.5, colors='k', ls='--')
ax.hlines(y[j], [0], x[j], alpha=0.5, colors='k', ls='--')
plt.show()

For example, if we imagine these draws as being observations of wealth across a
sample of households, then the dashed lines show that the bottom 80\% of
households own just over 40\% of total wealth.

**Exercise**

Using the definition of the Lorenz curve given above and NumPy, try to write
your own version of `qe.lorenz_curve`.  

* If possible, accelerate your code with Numba
* Try to replicate the figure above.

In [ ]:
# Put your code here

In [ ]:
for i in range(12):
    print("Solution below.")

**Solution**

In [ ]:
@numba.jit
def lorenz_curve(w):
    n = len(w)
    w = np.sort(w)
    s = np.zeros(n + 1)
    s[1:] = np.cumsum(w)  # s[i] = sum_{j <= i} w_j
    x = np.zeros(n + 1)
    y = np.zeros(n + 1)
    for i in range(1, n + 1):
        x[i] = i / n
        y[i] = s[i] / s[n]
    return x, y

fig, ax = plt.subplots()
ax.plot(x, y, label=f'lognormal sample', lw=2)
ax.plot(x, x, label='equality', lw=2)
ax.legend(fontsize=12)
ax.set_ylim((0, 1))
ax.set_xlim((0, 1))
j = 1600  # dashed lines for j-th element
ax.vlines(x[j], [0.0], y[j], alpha=0.5, colors='k', ls='--')
ax.hlines(y[j], [0], x[j], alpha=0.5, colors='k', ls='--')
plt.show()


## The Gini coefficient

### Definition


Continuing to assume that $w_1, \ldots, w_n$ has been sorted from smallest to largest,
the Gini coefficient of the sample is defined by

\begin{equation}
    \label{eq:gini}
    G :=
    \frac
        {\sum_{i=1}^n \sum_{j = 1}^n |w_j - w_i|}
        {2n\sum_{i=1}^n w_i}.
\end{equation}



### Using quantecon's routine

Let's examine the Gini coefficient in some simulations using `gini_coefficient`
from `quantecon`.

The following code computes the Gini coefficients for five different populations.

Each of these populations is generated by drawing from a lognormal distribution with parameters $\mu$ (mean) and $\sigma$ (standard deviation).

To create the five populations, we vary $\sigma$ over a grid of length $5$
between $0.2$ and $4$.

In each case we set $\mu = - \sigma^2 / 2$, so that the mean of the distribution does not change with $\sigma$.

In [ ]:
k = 5
σ_vals = np.linspace(0.2, 4, k)
n = 2_000
ginis = []
for σ in σ_vals:
    μ = -σ**2 / 2
    y = np.exp(μ + σ * np.random.randn(n))
    ginis.append(qe.gini_coefficient(y))

In [ ]:
fig, ax = plt.subplots()
ax.plot(σ_vals, ginis, marker='o')
ax.set_xlabel('$\sigma$', fontsize=12)
ax.set_ylabel('Gini coefficient', fontsize=12)
plt.show()

The plots show that inequality rises with $\sigma$ (as measured by the Gini coefficient).

**Exercise**

Using the definition above and NumPy, try to write your own version of
`qe.gini_coefficient`.  

* Try to replicate the figure above.
* If possible, accelerate your code with Numba
* If possible, parallelize one of the loops

In [ ]:
# Put your code here

In [ ]:
for i in range(12):
    print("Solution below.")

**Solution**

Here's one solution.

Notice how easy it is to parallelize the loop --- even though `s` is common across the outer loops, which violates independence, this loop is still efficiently parallelized.

In [ ]:
@numba.jit(parallel=True)
def gini_coefficient(w):
    n = len(w)
    s = 0.0
    for i in numba.prange(n):
        for j in range(n):
            s += abs(w[i] - w[j])
    return s / (2 * n * np.sum(w))

In [ ]:
ginis = []

for σ in σ_vals:
    μ = -σ**2 / 2
    y = np.exp(μ + σ * np.random.randn(n))
    ginis.append(gini_coefficient(y))


fig, ax = plt.subplots()
ax.plot(σ_vals, ginis, marker='o')
ax.set_xlabel('$\sigma$', fontsize=12)
ax.set_ylabel('Gini coefficient', fontsize=12)
plt.show()